In [1]:
# Está celda se puede descomentar en caso de que no esté instalado alguno de las 
# librerías necesarias
# !pip install pandasql; pip install pandas

In [2]:
from collections import Counter
import re
import numpy as np
import pandas as pd
import pandasql as psql

<center>
    <img src="https://i.ibb.co/8DB9xryv/portada-t1.png" alt="portada-t1" border="0" />
</center>

# Parte 1

Como queremos saber de qué se habla en el Congreso, lo primero es mirar a los .csv llamados participacion descripcion. Hay una columna llamada TEXTO_PRINCIPAL y otra llamada
DESCRIPCION_DEBATE	, en las que se nos señala sobre qué habló un parlamentario en un momento determinado. La idea es que partas etiquetando los datos: para cada intervención nos interesa tener una o
más keyword que nos señala qué TEMAS se trató. Algunos ejemplos pueden ser “deportes”, “delincuencia”,
“medio ambiente”, FECHA entre otros. Además, si encuentras que los datos están sucios o necesitas hacer otro
procesamiento, puedes hacerlo.
Para la obtención de keywords puedes hacer lo que tú quieras, desde ver si el TEXTO_PRINCIPAL contiene
una palabra definida por ti, hasta correr un modelo de lenguaje que lea automáticamente la intervención
y obtenga keywords. En el futuro vas a tener que agrupar por estos tópicos, por lo que es importante que
cada tópico se identifique únicamente. Por ejemplo, puedes tener problemas si tienes la keyword “deporte”
y también “deportes”.

In [3]:
# Se puede cambiar los path a sus archivos, pero estos se conectan a donde los tengo guardados en mi repositorio

path_to_2023_1 = 'https://raw.githubusercontent.com/Fernanda-Bley/IIC2440/refs/heads/main/T1/participacion/participacion/participacion_descripcion_2023_1.csv'
path_to_2023_2 = 'https://raw.githubusercontent.com/Fernanda-Bley/IIC2440/refs/heads/main/T1/participacion/participacion/participacion_descripcion_2023_2.csv'
path_to_2024 = 'https://raw.githubusercontent.com/Fernanda-Bley/IIC2440/refs/heads/main/T1/participacion/participacion/participacion_descripcion_2024.csv'
path_to_info = 'https://raw.githubusercontent.com/Fernanda-Bley/IIC2440/refs/heads/main/T1/participacion/participacion/parlamentarios_info_general.csv'

descripcion_2023_1 = pd.read_csv(path_to_2023_1)
descripcion_2023_2 = pd.read_csv(path_to_2023_2)
descripcion_2024 = pd.read_csv(path_to_2024)
info_parlamento = pd.read_csv(path_to_info)

In [4]:
# Vamos a recuperar solo las columnas que queremos estudiar, para eso definimos queries

first2023 = """
SELECT TIPO_SESION, TIPO_PARTICIPACION, CAMARA, TEXTO_PRINCIPAL, TEXTO_ANTECEDENTE, DESCRIPCION_DEBATE
FROM descripcion_2023_1
"""

second2023 = """
SELECT TIPO_SESION, TIPO_PARTICIPACION, CAMARA, TEXTO_PRINCIPAL, TEXTO_ANTECEDENTE, DESCRIPCION_DEBATE
FROM descripcion_2023_2
"""

all2024 = """
SELECT TIPO_SESION, TIPO_PARTICIPACION, CAMARA, TEXTO_PRINCIPAL, TEXTO_ANTECEDENTE, DESCRIPCION_DEBATE
FROM descripcion_2024
"""



In [5]:
psql.sqldf(all2024)["TEXTO_PRINCIPAL"]

0        La señora ALLENDE.- \nGracias, Presidente. \nH...
1        La señora ALLENDE.- \nGracias, Presidente. \nC...
2        La señora ALLENDE.- \nGracias, Presidente. \nP...
3        La señora ALLENDE.- \nPresidente , yo le pedí ...
4        La señora ALLENDE.- \nPresidente, como ha seña...
                               ...                        
21866    El señor CIFUENTES (Presidente).- \n \n Corres...
21867    La Diputada Yeomans (Presidenta) reconoció la ...
21868    -DiputadaYeomans ,   doña Gael . Acciones dest...
21869    -DiputadaYeomans ,   doña Gael . Acciones dest...
21870    El señor CIFUENTES (Presidente).- \n \n Corres...
Name: TEXTO_PRINCIPAL, Length: 21871, dtype: object

Investiguemos ``¿Cuales son los problemas de los que más se hablan cada año?``
Para filtrar las palabras comunes crearemos un filtro que incluya temas de las que más se hablan en el senado según el informe de [Febrero 2023](https://www.senado.cl/comunicaciones/noticias/pulso-legislativo-271-proyectos-despachados-y-casi-1300-sesiones-de-sala-y) y [Enero 2025](https://www.senado.cl/comunicaciones/noticias/los-temas-que-vienen-comisiones-retoman-su-trabajo-con-intensa-agenda-de): ``seguridad, crímenes, elecciones, pensiones, vivienda, impuestos, salud, educación, transporte, economía, inflación, COVID-19 y medio ambiente``

In [6]:
# Creamos esta función para que nos retorne los temas más comunes
def CommonSubjects(DataSet):
    filtros = ["pensión", "elecciones", "crímenes", "vivienda", "impuestos","combustibles", "seguridad", "salud", "educación", "transporte", "economía", "inflación", "COVID-19", "medio ambiente"]
    text_column = 'TEXTO_PRINCIPAL'
    all_text = ' '.join(psql.sqldf(DataSet)[text_column].astype(str))
    
    words = re.findall(r'\w+', all_text.lower())
    word_counts = Counter(words)
    
    
    subjects = {key: value for key, value in word_counts.items() if key in filtros}
    top_subjects = dict(sorted(subjects.items(), key=lambda item: item[1], reverse=True))

    return top_subjects

In [7]:
# Por el tamaño de los dataset, se demora aprox 10 segundos en correr
common_subjects = [CommonSubjects(first2023), CommonSubjects(second2023), CommonSubjects(all2024)]
years = ["2023-1", "2023-2", "2024"]

In [8]:
for year, common in zip(years, common_subjects):
    print(f"En el periodo {year} los TEMASSSs más comunes fueron:\n", ", ".join(list(common.keys())))

En el periodo 2023-1 los TEMASSSs más comunes fueron:
 seguridad, salud, educación, vivienda, impuestos, transporte, economía, pensión, inflación, combustibles, crímenes, elecciones
En el periodo 2023-2 los TEMASSSs más comunes fueron:
 salud, seguridad, educación, vivienda, transporte, economía, crímenes, impuestos, pensión, elecciones, combustibles, inflación
En el periodo 2024 los TEMASSSs más comunes fueron:
 seguridad, salud, educación, transporte, vivienda, economía, impuestos, elecciones, pensión, crímenes, combustibles, inflación


Los temas se mantienen de manera consistente así que mantendré estas *keywords*: 
``seguridad, salud, educación, transporte, vivienda, economía, elecciones, pensión, crímenes, combustibles, inflación``

# Parte 2 - [1 pts] - Modelamiento
Para la modelación, queremos que entiendas las entidades y tengas presente las consultas que vas a
realizar. Primero, queremos partir de un diagrama normalizado que tenga separado las distintas entidades, que serı́an los parlamentarios, partidos polı́ticos, debates, intervenciones, y las keywords que calculaste. Una vez contar con un modelo normalizado, puedes crear las vistas que estimes convenientes.

Luego, ten presente que harás las siguientes consultas:
- Cuáles son las top 5 temáticas más tratadas, para cada mes en el que hay registros.
- Media móvil de intervenciones por partido polı́tico con un intervalo de 3 meses.
- Para cada trimestre, ver el TEMAS principal tratado por cada partido polı́tico.
- Poder pararse en un mes y ver cuál es el top 3 de temáticas tratada por cada partido. Esto vas a querer repetirlo para algunos meses.

In [9]:
# Los parlamentarios, debates e intervenciones.

first2023 = """
SELECT PARLAMENTARIO_ID, TIPO_SESION, FECHA, TIPO_PARTICIPACION, TEXTO_PRINCIPAL, TEXTO_ANTECEDENTE, DESCRIPCION_DEBATE
FROM descripcion_2023_1
"""

second2023 = """
SELECT PARLAMENTARIO_ID, TIPO_SESION, FECHA, TIPO_PARTICIPACION, TEXTO_PRINCIPAL, TEXTO_ANTECEDENTE, DESCRIPCION_DEBATE
FROM descripcion_2023_2
"""

all2024 = """
SELECT PARLAMENTARIO_ID, TIPO_SESION, FECHA, TIPO_PARTICIPACION, TEXTO_PRINCIPAL, TEXTO_ANTECEDENTE, DESCRIPCION_DEBATE
FROM descripcion_2024
"""

parlamento = """
SELECT PARLAMENTARIO_ID, PARTIDO_POLITICO
FROM info_parlamento
"""

f2023_1 = psql.sqldf(first2023)
s2023_1 = psql.sqldf(second2023)
a2024 = psql.sqldf(all2024)
id_politico = psql.sqldf(parlamento)

In [10]:
# Añadir: keyword, meses, trimestres y partidos polı́ticos.

# keyword
def assign_subject(text, filtros):
    # Convertir el texto a minúsculas y dividir en palabras
    words = re.findall(r'\w+', text.lower())
    # Encontrar los temas que están en el texto usando un conjunto para evitar duplicados
    found_subjects = set(word for word in words if word in filtros)
    # Retornar los temas encontrados como lista o None si no hay ninguno
    return list(found_subjects) if found_subjects else "otros"

# trimestre
def asignar_trimestre(mes):
    if 1 <= mes <= 4:
        return 1  
    elif 5 <= mes <= 8:
        return 2
    elif 9 <= mes <= 12:
        return 3
    
# Añadir partido político
f2023_1_with_party = psql.sqldf("""
SELECT f.*, p.PARTIDO_POLITICO
FROM f2023_1 f
LEFT JOIN id_politico p ON f.PARLAMENTARIO_ID = p.PARLAMENTARIO_ID
""")


s2023_1_with_party = psql.sqldf("""
SELECT s.*, p.PARTIDO_POLITICO
FROM s2023_1 s
LEFT JOIN id_politico p ON s.PARLAMENTARIO_ID = p.PARLAMENTARIO_ID
""")


a2024_with_party = psql.sqldf("""
SELECT a.*, p.PARTIDO_POLITICO
FROM a2024 a
LEFT JOIN id_politico p ON a.PARLAMENTARIO_ID = p.PARLAMENTARIO_ID
""")

In [11]:
## Se demora 8 segundos aprox para que corra el código
# Definir los filtros
filtros = ["seguridad", "salud", "educación", "transporte", "vivienda", "economía", "elecciones", "pensión", "crímenes", "combustibles", "inflación"]

# Añadiendo keywords
f2023_1_with_party['TEMAS'] = f2023_1_with_party['TEXTO_PRINCIPAL'].apply(lambda x: assign_subject(x, filtros))
s2023_1_with_party['TEMAS'] = s2023_1_with_party['TEXTO_PRINCIPAL'].apply(lambda x: assign_subject(x, filtros))
a2024_with_party['TEMAS'] = a2024_with_party['TEXTO_PRINCIPAL'].apply(lambda x: assign_subject(x, filtros))

# Añadiendo meses
f2023_1_with_party["MES"] = pd.to_datetime(f2023_1_with_party["FECHA"]).dt.month_name()
s2023_1_with_party["MES"] = pd.to_datetime(s2023_1_with_party["FECHA"]).dt.month_name()
a2024_with_party["MES"] = pd.to_datetime(a2024_with_party["FECHA"]).dt.month_name()

# Añadiendo trimestres
f2023_1_with_party["TRIMESTRE"] = pd.to_datetime(f2023_1_with_party['FECHA']).dt.month.apply(lambda x: asignar_trimestre(x))
s2023_1_with_party["TRIMESTRE"] = pd.to_datetime(s2023_1_with_party['FECHA']).dt.month.apply(lambda x: asignar_trimestre(x))
a2024_with_party["TRIMESTRE"] = pd.to_datetime(a2024_with_party['FECHA']).dt.month.apply(lambda x: asignar_trimestre(x))



In [12]:
# Vamos a quedarnos a unir todo con las columnas que nos sirven y exportamos los csv

unidos = pd.concat([f2023_1_with_party, s2023_1_with_party], ignore_index=True)
unidos[['PARTIDO_POLITICO', 'TEMAS', 'MES', 'TRIMESTRE']].to_csv('total_2023.csv', index=False)
a2024_with_party[['PARTIDO_POLITICO', 'TEMAS', 'MES', 'TRIMESTRE']].to_csv('total_2024.csv', index=False)